In [ ]:
"""
Imported Libraries
"""
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, UpSampling2D
from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import ZeroPadding2D
from keras.models import Sequential, load_model
from keras.regularizers import l1 , l2 , l1_l2
from keras.optimizers import Adam , SGD
from keras import regularizers
import matplotlib.pylab as plt
import numpy as np
import datetime
import keras
import glob
import cv2

In [ ]:
"""
Model Variables
"""
input_shape = 32
no_classes = 23

In [ ]:
"""
Model Layers
"""
classifier = Sequential()
layer_00 = Conv2D(6, (5, 5),input_shape = (input_shape,input_shape, 3), activation = 'relu')
classifier.add(layer_00) 
classifier.add(BatchNormalization(axis=-1))

layer_00_w = layer_00.get_weights()[0][:,:,0,:]
for i in range(1,26):
    plt.subplot(5,5,i)
    plt.imshow(layer_00_w[:,:,i],interpolation="nearest",cmap="gray")
plt.show()



layer_01 = Conv2D(32, (3, 3), activation = 'relu')
classifier.add(layer_01)
classifier.add(BatchNormalization(axis=-1))

layer_01_w = layer_01.get_weights()[0][:,:,0,:]
for i in range(1,26):
    plt.subplot(5,5,i)
    plt.imshow(layer_01_w[:,:,i],interpolation="nearest",cmap="gray")
plt.show()


classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Conv2D(64,(3, 3), activation = 'relu'))
classifier.add(BatchNormalization(axis=-1))
classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
classifier.add(BatchNormalization(axis=-1))
classifier.add(MaxPooling2D(pool_size=(2,2)))

cnn_outdim = classifier.layers[len(classifier.layers) - 1].output.get_shape()[1]
print(cnn_outdim)

classifier.add(Flatten())
classifier.add(Dense(output_dim = 120, activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.2))
classifier.add(Dense(output_dim = 84, activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.2))
classifier.add(Dense(output_dim = no_classes, activation = 'softmax'))


"""
classifier = Sequential()

classifier.add(Conv2D(filters=8, kernel_size=2, input_shape=(224, 224, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=2, data_format='channels_last'))
classifier.add(Conv2D(filters=16, kernel_size=2, activation='relu'))
classifier.add(MaxPooling2D(pool_size=2, data_format='channels_last'))
classifier.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
classifier.add(MaxPooling2D(pool_size=2, data_format='channels_last'))
classifier.add(GlobalAveragePooling2D())
classifier.add(Dense(no_classes, activation='relu'))
"""

In [ ]:
"""
Compile variables and Data Generation
"""

#sgd = SGD (lr = 0.001  , nesterov = True)
adam = Adam (lr=0.001)
#classifier.compile(loss = 'categorical_crossentropy',  metrics = ["accuracy"], optimizer = Adam(lr=1e-3))
classifier.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = adam)

train_datagen = ImageDataGenerator(rotation_range=8, width_shift_range=0.1, shear_range=0.3,
                         height_shift_range=0.1, zoom_range=0.08)
test_datagen = ImageDataGenerator( )

#CLOUD PATH
#dir_train = '/job2/train'
#dir_test =  '/job2/test'

#Desktop PATH
dir_train = 'train'
dir_test =  'test'

training_set = train_datagen.flow_from_directory(dir_train,
                                                 target_size = (input_shape,input_shape),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(dir_test,
                                            target_size = (input_shape,input_shape),
                                            batch_size = 64,
                                            class_mode = 'categorical')

In [ ]:
"""
Train Model
"""

class test(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        self.letters_list = ['beh' , 'teh' , 'seh' , 'gem' , 'ha' , 'kha' ,
                             'sen' , 'shen' , 'sad' , 'dad' , 'tah' , 'zah' ,
                             'ayen' , 'ghen', 'feh' , 'kaaf' , 'kaf' , 'lam',
                             'mem' , 'non' , 'heh' , 'yeh' , 'hamza']
        self.rightl = [15,17,6,21,1,15,6,21,10,10,21,3,21,17,
                       18,4,18,18,4,12,21,17,18,19,8,2,17,17,21,19,15]
        self.outputlist = []
        self.predicted = []
        self.cnt = 0
        for file in glob.glob("*.png"):
            self.img = cv2.imread(file)
            self.imresize = cv2.resize(self.img, (input_shape , input_shape))
            self.imresize = cv2.bitwise_not(self.imresize)
            self.imresize = cv2.cvtColor(self.imresize , cv2.COLOR_BGR2GRAY)
            ret,self.thresh1 = cv2.threshold(self.imresize,127,255,cv2.THRESH_BINARY)
            self.thresh1 = cv2.cvtColor(self.thresh1, cv2.COLOR_GRAY2BGR)
            self.imlist = np.array([self.thresh1])
            self.predhis = (classifier.predict_classes(self.imlist))
            self.outputlist.append(self.predhis[0])
        for i in range (0 , len(self.rightl)-1):
            if self.outputlist[i] == self.rightl[i]:
                self.cnt += 1
                self.predicted.append(self.outputlist[i])

        print(self.cnt)
        print("\n")
        print(self.predicted)
        self.now =  datetime.datetime.now()
        self.y = str(self.now.date().year)
        self.m = str(self.now.date().month)
        self.d = str(self.now.date().day)
        self.h = str(self.now.hour)
        self.mi = str(self.now.minute)
        self.name = "Dvx_"+self.y+self.m+self.d+"_"+self.h+self.mi+" - "+str(self.cnt)
        #CLOUD PATH
        #self.dir_model = "/output/"+self.name+".h5"
        #Desktop PATH
        self.dir_model = self.name+".h5"
        classifier.save(self.dir_model)

predictions=test()
history = classifier.fit_generator(training_set,
                         samples_per_epoch = 17843,
                         nb_epoch = 100,
                         validation_data = test_set,
                         nb_val_samples = 4686,
                         callbacks=[predictions])

In [ ]:
"""
Save Model
"""

now =  datetime.datetime.now()
y = str(now.date().year)
m = str(now.date().month)
d = str(now.date().day)
h = str(now.hour)
mi = str(now.minute)
s = str(now.second)
name = "Dvx_"+y+m+d+"_"+h+mi

#CLOUD PATH
#dir_model = "/output/"+name+".h5"
#Desktop PATH
dir_model = name+".h5"

classifier.save(dir_model)

In [ ]:
"""
Plot Acc & Loss
"""

plt.figure(1)
plt.subplot(211)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.subplot(212)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
#plt.show()
plt.savefig("/output/"+name+".png")
 

In [ ]:
#
#
# """
# Model Validation
# """
def original_and_inverted (file, shape, obj_list, Model):
    img = cv2.imread(file)
    imresize = cv2.resize(img, (shape , shape))
    
    ret,thresh0 = cv2.threshold(imresize.copy(),127,255,cv2.THRESH_BINARY)
    imlist = np.array([thresh0])
    classes0 =Model.predict_classes(imlist)

    img_inv = cv2.bitwise_not(imresize.copy())
    ret,thresh1 = cv2.threshold(img_inv.copy(),127,255,cv2.THRESH_BINARY)
    imlist = np.array([thresh1])
    classes1 =Model.predict_classes(imlist)

    print("Original Image: "+obj_list[classes0[0]]+"  Inverted Image: "+obj_list[classes1[0]] )


letters_list = ['beh' , 'teh' , 'seh' , 'gem' , 'ha' , 'kha' ,
                  'sen' , 'shen' , 'sad' , 'dad' , 'tah' , 'zah' ,
                  'ayen' , 'ghen', 'feh' , 'kaaf' , 'kaf' , 'lam',
                  'mem' , 'non' , 'heh' , 'yeh' , 'hamza']

classifier = load_model("Dvx_20171221_2058 - 17.h5")
classifier.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')
 
 
original_and_inverted("30.png", input_shape, letters_list, classifier)